# homework6

## Name: Yiming Yan

## USCID: 9932750243

### (a) Download the Anuran Calls (MFCCs) Data Set. Choose 70% of the data randomly as the training set.

In [1]:
import pandas as pd
import random
import numpy as np
import math

In [2]:
def split_data(o_data,train_per=0.7):
    data=o_data.copy()
    columns=data.columns
    training_data=pd.DataFrame(columns=columns)
    test_data=pd.DataFrame(columns=columns)
    length=len(data.values)
    l=round(length*train_per)
    ran_num=range(length)
    i=0
    while i<1:
        k=random.choice(ran_num)
        index=list(training_data.index)
        train_length=len(training_data.values)
        if train_length==l:
            break
        else:
          if k in index:
             continue
          else:
            training_data=training_data.append(data.loc[k])
            data.drop([k],inplace=True)
    test_data=data
    training_data.index=range(len(training_data)) 
    test_data.index=range(len(test_data))
    return training_data,test_data

In [3]:
origin_set=pd.DataFrame(pd.read_csv('G:/EE559/hw/hw6/Frogs_MFCCs.csv'))
training_set,test_set=split_data(origin_set)

In [30]:
training_set

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.000000,0.331730,0.018946,0.398650,0.400487,0.104183,-0.227688,-0.090622,0.164318,0.255465,...,-0.184352,0.069889,0.165210,0.011003,-0.069136,-0.069716,Dendrobatidae,Ameerega,Ameeregatrivittata,11
1,1.000000,0.155049,0.125537,0.576056,0.232781,-0.047139,-0.089704,0.156122,0.200915,-0.064862,...,0.088486,0.006328,-0.062312,-0.074370,0.078301,0.128258,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,18
2,1.000000,0.308728,0.234934,0.527024,0.178472,0.009294,-0.140521,0.110709,0.326415,-0.014896,...,0.161011,0.055237,-0.063321,-0.115348,0.097101,0.229945,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,23
3,1.000000,0.447066,0.398681,0.512818,0.064754,0.001957,-0.033550,0.078006,0.232600,-0.001685,...,0.196299,-0.043328,-0.045204,-0.019360,0.087126,0.160549,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,19
4,1.000000,0.118416,0.336452,0.287914,0.163894,0.220670,0.062985,-0.162259,-0.030521,0.093829,...,0.089309,0.002693,0.000863,-0.042927,-0.107604,0.003791,Hylidae,Hypsiboas,HypsiboasCordobae,41
5,1.000000,0.235832,0.353584,0.262535,0.126709,0.186260,0.072340,-0.100870,-0.004227,0.069698,...,0.068307,-0.017618,-0.008540,-0.005693,-0.022963,0.032874,Hylidae,Hypsiboas,HypsiboasCordobae,40
6,1.000000,0.394704,0.420662,0.352724,0.046914,0.105684,0.095945,-0.040442,0.043874,0.173450,...,0.022360,-0.006503,0.017768,0.012500,-0.029048,-0.026501,Hylidae,Hypsiboas,HypsiboasCordobae,41
7,1.000000,0.240077,0.219120,0.574663,0.235800,0.003970,-0.101245,0.079260,0.292515,0.118585,...,0.232252,0.036825,-0.046847,-0.126784,-0.086751,0.181513,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,22
8,1.000000,0.397913,0.275841,0.455123,0.092774,0.024469,-0.061071,0.092801,0.289208,-0.006060,...,0.182672,-0.002742,-0.148895,-0.170784,0.100100,0.166811,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,14
9,1.000000,0.260914,0.195402,0.497887,0.168214,0.036571,-0.090312,0.064087,0.289242,0.018740,...,0.197074,0.109075,-0.053968,-0.164694,0.038401,0.202065,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,23


### (b)

#### i. Research exact match and hamming score/ loss methods for evaluating multilabel classication and use them in evaluating the classifiers in this problem.

Accuracy score/ Exact match metric: This function calculates subset accuracy meaning the predicted set of labels should exactly match with the true set of labels.Exact match is the most strict metric, indicating the percentage of samples that have all their labels classified correctly.

Hamming loss is the fraction of the wrong labels to the total number of labels.

In [4]:
def exact_match(y_true,y_predict):
    length,width=y_true.shape
    count=0
    for true,predict in zip(y_true,y_predict):
        true=list(true)
        predict=list(predict)
        if true==predict:
            count+=1
    fraction=count/length
    return fraction



#### ii. Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation. You are welcome to try to solve the problem with both normalized and raw attributes and report the results.

In [5]:
from sklearn.model_selection  import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss

In [6]:
def extrate_data(data):
    features=np.array(data.drop(['Family', 'Genus', 'Species', 'RecordID'], axis=1))
    label1=np.array(data['Family'])
    label2=np.array(data['Genus'])
    label3=np.array(data['Species'])
    return features,label1,label2,label3

In [7]:
# Gaussian Kernel width and Parameter C
sigma=[i/10 for i in range(1,21,1)]
gamma=[1/(2*sig*sig) for sig in sigma]
param_grid={'kernel':['rbf'],'C':[0.01,0.1,1,10,100],'gamma':gamma}


In [8]:
print(param_grid['gamma'])

[49.99999999999999, 12.499999999999998, 5.555555555555555, 3.1249999999999996, 2.0, 1.3888888888888888, 1.0204081632653064, 0.7812499999999999, 0.6172839506172839, 0.5, 0.4132231404958677, 0.3472222222222222, 0.29585798816568043, 0.2551020408163266, 0.2222222222222222, 0.19531249999999997, 0.17301038062283738, 0.15432098765432098, 0.13850415512465375, 0.125]


In [9]:
training_data,training_family,training_genus,training_species=extrate_data(training_set)
test_data,test_family,test_genus,test_species=extrate_data(test_set)

In [10]:
svc1 = SVC()
svc2 = SVC()
svc3 = SVC()
family_svc = GridSearchCV(svc1,param_grid,cv = 10,refit=True, n_jobs=-1)
genus_svc = GridSearchCV(svc2,param_grid,cv = 10,refit=True,n_jobs=-1)
species_svc = GridSearchCV(svc3,param_grid,cv = 10,refit=True,n_jobs=-1)   



In [11]:
family_model=family_svc.fit(training_data,training_family)
family_pre = family_model.predict(test_data)
wFamily = family_model.best_params_
hammingloss1=hamming_loss(test_family,family_pre)

In [12]:
genus_model=genus_svc.fit(training_data,training_genus)
genus_pre = genus_model.predict(test_data)
wGenus = genus_model.best_params_
hammingloss2=hamming_loss(test_genus,genus_pre)

In [13]:
species_model=species_svc.fit(training_data,training_species)
species_pre =species_model.predict(test_data)
wSpecies = species_model.best_params_

label_pre = np.array([family_pre, genus_pre, species_pre]).T
label_true= np.array([test_family, test_genus,test_species]).T

hammingloss3=hamming_loss(test_species,species_pre)
hammingloss=(hammingloss1+hammingloss2+hammingloss3)/3

exactmatch=exact_match(label_true,label_pre)

In [14]:
print(" penalty and width for Family: ",wFamily['C']," The gamma is",wFamily['gamma'],
      "The sigma is",math.sqrt(1/(2*wFamily['gamma'])))
print("penalty and width for Genus: ",wGenus['C']," The gamma is",wGenus['gamma'],
      "The sigma is",math.sqrt(1/(2*wGenus['gamma'])))
print("penalty and width for Species: ",wSpecies['C']," The gamma is",wSpecies['gamma'],
      "The sigma is",math.sqrt(1/(2*wSpecies['gamma'])))
print("exact match", exactmatch)
print("hamming loss",hammingloss)

 penalty and width for Family:  10  The gamma is 3.1249999999999996 The sigma is 0.4
penalty and width for Genus:  100  The gamma is 1.3888888888888888 The sigma is 0.6
penalty and width for Species:  10  The gamma is 2.0 The sigma is 0.5
exact match 0.9911996294580825
hamming loss 0.0057125212289640265


#### iii. Repeat 1(b)ii with L1-penalized SVMs. Remember to normalize the attributes.

According to the description "Mel-frequency cepstral coefficients (MFCCs) are coefficients that collectively make up an mel-frequency cepstrum (MFC). Due to each syllable has different length, every row (i) was normalized acording to MFCCs_i/(max(abs(MFCCs_i)))." , the whole data has been normalized.
So, I don't need to normalize the data.

In [15]:
from sklearn.svm import LinearSVC

In [16]:
params = {'C':[0.001,0.01,0.1,1,10,100,1000]}

In [17]:
lsvc1 = LinearSVC(penalty='l1', dual=False)
lsvc2 = LinearSVC(penalty='l1', dual=False)
lsvc3 = LinearSVC(penalty='l1', dual=False)
family_svcl=GridSearchCV(lsvc1,params,cv = 10,refit=True,n_jobs=-1)
genus_svcl= GridSearchCV(lsvc2,params,cv = 10,refit=True,n_jobs=-1)
species_svcl= GridSearchCV(lsvc3,params,cv = 10,refit=True,n_jobs=-1)

In [18]:
family_modell = family_svcl.fit(training_data,training_family)
family_prel = family_modell.predict(test_data)
wFamilyl = family_modell.best_params_ 
hamminglossl1=hamming_loss(test_family,family_prel)

In [19]:
genus_modell=genus_svcl.fit(training_data,training_genus)
genus_prel = genus_modell.predict(test_data)
wGenusl=genus_modell.best_params_
hamminglossl2=hamming_loss(test_genus,genus_prel)

In [20]:
species_modell = species_svcl.fit(training_data,training_species)
species_prel =species_modell.predict(test_data)
wSpeciesl = species_modell.best_params_

label_prel = np.array([family_prel, genus_prel, species_prel]).T
label_true= np.array([test_family, test_genus,test_species]).T

hamminglossl3=hamming_loss(test_species,species_prel)
hamminglossl=(hamminglossl1+hamminglossl2+hamminglossl3)/3

exactmatch=exact_match(label_true,label_prel)

In [21]:
print(" penalty and width for Family: ",wFamilyl['C'])
print("penalty and width for Genus: ",wGenusl['C'])
print("penalty and width for Species: ",wSpeciesl['C'])
print("exact match", exactmatch)
print("hamming loss",hamminglossl)

 penalty and width for Family:  100
penalty and width for Genus:  100
penalty and width for Species:  100
exact match 0.9212598425196851
hamming loss 0.04616334722865525


#### iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classifiers you trained.

In [22]:
from imblearn.over_sampling import SMOTE

In [23]:
params = {'C':[0.001,0.01, 0.1,1,10,100,1000]}

In [24]:
sm=SMOTE()
family_data_res,family_label_res = sm.fit_sample(training_data, training_family)
genus_data_res,genus_label_res = sm.fit_sample(training_data, training_genus)
species_data_res,species_label_res = sm.fit_sample(training_data, training_species)

In [25]:
lsvc1 = LinearSVC(penalty='l1', dual=False)
lsvc2 = LinearSVC(penalty='l1', dual=False)
lsvc3 = LinearSVC(penalty='l1', dual=False)
family_svcl=GridSearchCV(lsvc1,params,cv = 10,refit=True,n_jobs=-1)
genus_svcl= GridSearchCV(lsvc2,params,cv = 10,refit=True,n_jobs=-1)
species_svcl= GridSearchCV(lsvc3,params,cv = 10,refit=True,n_jobs=-1)

In [26]:
family_modell = family_svcl.fit(family_data_res,family_label_res)
family_prel = family_modell.predict(test_data)
wFamilyl = family_modell.best_params_ 
hamminglossl1=hamming_loss(test_family,family_prel)

In [27]:
genus_modell=genus_svcl.fit(genus_data_res,genus_label_res)
genus_prel = genus_modell.predict(test_data)
wGenusl=genus_modell.best_params_
hamminglossl2=hamming_loss(test_genus,genus_prel)

In [28]:
species_modell = species_svcl.fit(species_data_res,species_label_res)
species_prel =species_modell.predict(test_data)
wSpeciesl = species_modell.best_params_

label_prel = np.array([family_prel, genus_prel, species_prel]).T
label_true= np.array([test_family, test_genus,test_species]).T

hamminglossl3=hamming_loss(test_species,species_prel)
hamminglossl=(hamminglossl1+hamminglossl2+hamminglossl3)/3

exactmatch=exact_match(label_true,label_prel)

In [29]:
print(" penalty and width for Family: ",wFamilyl['C'])
print("penalty and width for Genus: ",wGenusl['C'])
print("penalty and width for Species: ",wSpeciesl['C'])
print("exact match", exactmatch)
print("hamming loss",hamminglossl)

 penalty and width for Family:  100
penalty and width for Genus:  1000
penalty and width for Species:  100
exact match 0.8679944418712366
hamming loss 0.06824146981627296
